In [1]:
import dotenv
import jupyter_black

dotenv.load_dotenv()
jupyter_black.load()

In [2]:
from typing import Any


from baml_agents import (
    with_hooks,
    OnBeforeCallHookContext,
    OnBeforeCallHookSync,
    get_args_block_str,
)
from baml_client import types
from baml_client.sync_client import b


def generate_baml_test(test_name, baml_function_name, params):
    return f"""\
test {test_name} {{
  functions [{baml_function_name}]
  args {{
{get_args_block_str(params)}
  }}
}}\n\n"""


class BamlTestGeneratorHook(OnBeforeCallHookSync):
    def __init__(self):
        self.counter = 0
        self.baml_test_source_code = ""

    def on_before_call(self, params: dict[str, Any], ctx: OnBeforeCallHookContext):
        self.counter += 1
        test_name = f"llm_call_{self.counter}"
        self.baml_test_source_code += generate_baml_test(
            test_name, ctx.baml_function_name, params
        )

In [3]:
test_hook = BamlTestGeneratorHook()
b = with_hooks(b, [test_hook])

b.ChatWithLLM(
    messages=[
        types.LLMChatMessage(role="user", content="What's the weather like?"),
    ]
)
b.request.ChatWithLLM(
    messages=[
        types.LLMChatMessage(role="user", content="What's the weather like?"),
        types.LLMChatMessage(role="assistant", content="It's sunny."),
        types.LLMChatMessage(role="user", content="Do you like sunny weather?"),
    ]
)

print(test_hook.baml_test_source_code)

test llm_call_1 {
  functions [ChatWithLLM]
  args {
    messages [
      {
        role #"user"#
        content #"What's the weather like?"#
      }
    ]
  }
}

test llm_call_2 {
  functions [ChatWithLLM]
  args {
    messages [
      {
        role #"user"#
        content #"What's the weather like?"#
      },
      {
        role #"assistant"#
        content #"It's sunny."#
      },
      {
        role #"user"#
        content #"Do you like sunny weather?"#
      }
    ]
  }
}


